<a href="https://colab.research.google.com/github/DavidPachis/dataScience/blob/main/taller1/notebooks/taller1_DV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1 - Análisis de datos

MINE-4101: Applied Data Science  
Univerisdad de los Andes
David Augusto Vásquez Pachón  
código: 201512574  
Dataset: Spotify data /(Data_Taller_1.csv)
  
Last update: August 30, 2022

In [ ]:
!pip install --upgrade pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [ ]:
# not installing last version due lacking of a couple of libraries removed
#in last version 
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


In [ ]:
# Importing libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
# This line is to see graphs's outputs below cells 
%matplotlib inline

In [ ]:
# Pandas configuration for extending the number of rows and columns,
#to visualize if not limit set parameter to None or -1
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
# Loading the CSV file as dataframe

url = 'https://raw.githubusercontent.com/datascience-uniandes/eda-tutorial/master/data/fifa.csv'
fifa_df = pd.read_csv(url)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Showing dataframe dimensions
fifa_df.shape

(17981, 74)

In [ ]:
# Showing column types
fifa_df.dtypes

Name                    object
Age                      int64
Photo                   object
Nationality             object
Flag                    object
Overall                  int64
Potential                int64
Club                    object
Club Logo               object
Value                   object
Wage                    object
Special                  int64
Acceleration            object
Aggression              object
Agility                 object
Balance                 object
Ball control            object
Composure               object
Crossing                object
Curve                   object
Dribbling               object
Finishing               object
Free kick accuracy      object
GK diving               object
GK handling             object
GK kicking              object
GK positioning          object
GK reflexes             object
Heading accuracy        object
Interceptions           object
Jumping                 object
Long passing            object
Long sho

In [ ]:
# Showing first rows
fifa_df.head()

### Making some basic transformations for columns referring to money: *Value* and *Wage*

In [ ]:
# Analyzing unique values for columns Value and Wage
set(''.join(fifa_df['Value'].values + fifa_df['Wage'].values))

In [ ]:
# Transforming columns to float based on thousand and million symbols

def money_str_to_float(x):
    x = x.replace('€', '')             # Removing euro symbol
    if 'M' in x:                      # If value has million symbol, remove it and multiply by 1'000.000
        x = x.replace('M', '')
        return float(x) * 1000000
    elif 'K' in x:                    # If value has thousand symbol, remove it and multiply by 1.000
        x = x.replace('K', '')
        return float(x) * 1000
    else:                             # If any symbol is found, only transform the string to float
        return float(x)

fifa_df['Value'] = fifa_df['Value'].apply(money_str_to_float)
fifa_df['Wage'] = fifa_df['Wage'].apply(money_str_to_float)

In [ ]:
fifa_df.head()

### Analyzing the *Value* column

In [ ]:
# Getting the mean value for all players
fifa_df['Value'].mean()

In [ ]:
# Getting the median value for all players
fifa_df['Value'].median()

In [ ]:
# Getting the player with the highest value
max_value = fifa_df['Value'].max()

fifa_df.loc[fifa_df['Value'] == max_value]

In [ ]:
# Getting the main statistics for the Value column
fifa_df['Value'].describe()

In [ ]:
# Plotting the histogram for the column
fifa_df['Value'].hist()

In [ ]:
# Defining the number of bins and the scale for the y-axis for better visualizing
plt.figure()
fifa_df['Value'].hist(bins = 50)
plt.yscale('log')

In [ ]:
# Plotting distribution but using a boxplot
plt.figure(figsize = (18, 3))
plt.boxplot(fifa_df['Value'], vert = False)

In [ ]:
# Which are the players with outlier values?
fifa_df[fifa_df['Value'] > (fifa_df['Value'].mean() + fifa_df['Value'].std() * 3)]

In [ ]:
# Because the distribution is so skewed, a greatest number of standard deviations is used 
fifa_df[fifa_df['Value'] > (fifa_df['Value'].mean() + fifa_df['Value'].std() * 17)]

In [ ]:
# Using the method based on quartiles to select outliers
iqr = (fifa_df['Value'].quantile(.75) - fifa_df['Value'].quantile(.25))
fifa_df[fifa_df['Value'] > (fifa_df['Value'].quantile(.75) + iqr * 50)]

### Analyzing *Preferred Positions* column

In [ ]:
fifa_df['Preferred Positions'].unique()

<p style="color: red;">How to handle players with multiple Preferred Positions?</p>

<p style="color: red;">Let's take the position with the highest score!</p>

In [ ]:
# How many preferred positions the players have?

def count_pref_positions(x):
    return len(x.strip().split(' '))

fifa_df['Preferred Positions'].apply(count_pref_positions).value_counts(dropna = False, normalize = True)

In [ ]:
# Select the preferred position with highest score

def get_pref_position(row):
    pref_positions = row['Preferred Positions'].strip().split(' ')
    if len(pref_positions) == 1:
        return pref_positions[0]
    else:
        scores = np.array([row[p] for p in pref_positions])
        i_max = np.argmax(scores)
        return pref_positions[i_max]

fifa_df['Preferred Position'] = fifa_df.apply(get_pref_position, axis = 1)

In [ ]:
# Calculating absolute frecuency for preferred position
pref_positions_abs_frec = fifa_df['Preferred Position'].value_counts(dropna = False)
pref_positions_abs_frec

In [ ]:
# Plotting preferred position frecuency
# WARNING: line chart assumes continuity. It is a better choice when dealing with a continuous variable or time
pref_positions_abs_frec.plot()

In [ ]:
# Plotting again preferred position frecuency, using a horizontal bar chart this time
pref_positions_abs_frec.plot(kind = 'barh')

In [ ]:
# Calculating absolute frecuency and concatenating results
pref_positions_rel_frec = fifa_df['Preferred Position'].value_counts(dropna = False, normalize = True)
pd.concat([pref_positions_abs_frec, pref_positions_rel_frec], axis = 1)

### Analyzing *Age* vs. *Value*

In [ ]:
# Plotting a scatter plot for two quantitative variables
plt.figure(figsize = (13, 7))
plt.scatter(fifa_df['Age'], fifa_df['Value'], alpha = 0.5)
plt.title('Age vs. Value')
plt.xlabel('Age')
plt.ylabel('Value')
plt.show()

In [ ]:
# What is the (linear) correlation coefficient?
fifa_df[['Age', 'Value']].corr()

In [ ]:
# What is the (monotonic) correlation coefficient?
fifa_df[['Age', 'Value']].corr('spearman')

### Analyzing *Overall* vs. *Value*

In [ ]:
# Plotting a scatter plot for two quantitative variables
plt.figure(figsize = (13, 7))
plt.scatter(fifa_df['Overall'], fifa_df['Value'], alpha = 0.5, color = 'orange')
plt.title('Overall vs. Value')
plt.xlabel('Overall')
plt.ylabel('Value')
plt.show()

In [ ]:
# What is the (linear) correlation coefficient?
fifa_df[['Overall', 'Value']].corr()

In [ ]:
# What is the (monotonic) correlation coefficient?
fifa_df[['Overall', 'Value']].corr('spearman')

<p style="color: red;">Why both methods give different correlation coefficients for the same pair of variables?</p>

### Comparing among categories

In [ ]:
# Plotting wage by club
plt.figure(figsize = (13, 10))
sns.boxplot(x = 'Club', y = 'Wage', data = fifa_df.loc[fifa_df['Club'].isin(['Independiente Santa Fe', 'CD Los Millionarios Bogota'])])

In [ ]:
# Plotting wage by club, including a differential club
plt.figure(figsize = (13, 10))
sns.boxplot(x = 'Club', y = 'Wage', data = fifa_df.loc[fifa_df['Club'].isin(['Independiente Santa Fe', 'CD Los Millionarios Bogota', 'Real Madrid CF'])])

### Profiling the data

In [ ]:
profile = ProfileReport(fifa_df)

In [ ]:
profile.to_notebook_iframe()

In [ ]:
profile.to_file('report.html')